## **Compute risk bound and theta with Selection with Guaranteed Risk (SGR)**

Reference: https://arxiv.org/pdf/1705.08500 Algorithm 1

In [1]:
from risk_control import *
import numpy as np
import torch

## **Get Confidence Scores**

#### Method1 : **Maxprob**

In [2]:
from collections import Counter

def max_occurence(labels):
    count = Counter(labels)
    max_num = max(count, key=count.get)
    return max_num


    

In [18]:
data_path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/predictions/logits_and_labels/"
NUM_BATCH = 2139
residuals = []
kappa = []
softmax = torch.nn.Softmax(dim=1)
for batch_no in range(NUM_BATCH):
    file_name = "Logits_and_labels" + str(batch_no) + ".pt"
    data = torch.load(data_path + file_name)
    
    for logits, labels in zip(data['logits'], data['labels']):
        if(len(labels)) == 0:
            continue
        
        logits = torch.from_numpy(logits)
        prob = softmax(logits)
        # print(prob)
        # print(torch.max(prob).numpy())
        kappa.append(torch.max(prob).numpy())
        idx = torch.argmax(prob)
        # print(prob[0][idx])
        # print(labels)
        residuals.append(idx.item() != max_occurence(labels))
        # print(kappa)
        # print(residuals)
        # break
    # break
        
    

In [19]:
kappa = np.array(kappa)
residuals = np.array(residuals)
risk_dict_max_prob = {}
risk_stars = [0.01, 0.02, 0.1, 0.15, 0.20, 0.25,0.30, 1.0] # desired risk
delta = 0.001 ## confidence
for desired_risk in risk_stars:
    bound_cal = risk_control()
    [theta, b_star] = bound_cal.bound(desired_risk, delta, kappa, residuals, split= True)
    risk_dict_max_prob[str(desired_risk)] = [theta, b_star]


0.01 & 0.0078 & 0.2677 & 0.0080 & 0.2669 & 0.0100  \\
0.02 & 0.0173 & 0.3615 & 0.0176 & 0.3642 & 0.0200  \\
0.10 & 0.0957 & 0.6788 & 0.0959 & 0.6775 & 0.1000  \\
0.15 & 0.1453 & 0.8058 & 0.1464 & 0.8070 & 0.1500  \\
0.20 & 0.1950 & 0.9119 & 0.1926 & 0.9111 & 0.2000  \\
0.25 & 0.2441 & 1.0000 & 0.2445 & 1.0000 & 0.2492  \\
0.30 & 0.2450 & 1.0000 & 0.2436 & 1.0000 & 0.2502  \\
1.00 & 0.2453 & 1.0000 & 0.2434 & 1.0000 & 0.2504  \\


In [5]:
risk_dict_max_prob

{'0.02': [0.99085826, 0.019990195521006792],
 '0.1': [0.7031023, 0.0999923430674681],
 '0.15': [0.51198816, 0.14999186544652277],
 '0.2': [0.28016168, 0.19999890216158067],
 '0.25': [0.030047635, 0.24999369884229788],
 '0.3': [0.022310337, 0.24959095036491086],
 '1.0': [0.022311127, 0.24913977406440627]}

#### Method 2: **Vector Scaling**

In [6]:
import sys
sys.path.append("/teamspace/studios/this_studio/Selective_Prediction_VQA")
from calibration_methods import calibrator as cal

In [7]:
def load_calibrator(path, calibrator_type = "vector_calibrator", device = 'cpu'):
    dict = torch.load(path)
    if calibrator_type == "vector_calibrator":
        cali = cal.VectorScaling(bias=dict['biasFlag'], 
                                 weights= dict['weights'],
                                 num_label = dict['num_label'],
                                 device=device,
                                 print_verbose= False)
        cali.temperature = dict['temperature']
        cali.bias = dict['bias']
    else:
        #todo
        return None
    return cali

In [8]:
path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/calibration_methods/scaling/vector_calibrator.pt"
cali = load_calibrator(path, calibrator_type = "vector_calibrator", device='cuda')

In [9]:
data_path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/predictions/logits_and_labels/"
NUM_BATCH = 2139
residuals = []
kappa = []
softmax = torch.nn.Softmax(dim=1)
for batch_no in range(NUM_BATCH):
    file_name = "Logits_and_labels" + str(batch_no) + ".pt"
    data = torch.load(data_path + file_name)
    
    for logits, labels in zip(data['logits'], data['labels']):
        if(len(labels)) == 0:
            continue
        logits = cali.calibrate(logits)
        logits = torch.from_numpy(logits)
        
        prob = softmax(logits)
        # print(prob)
        # print(torch.max(prob).numpy())
        # prob.to('cpu')
        kappa.append(torch.max(prob).numpy())
        idx = torch.argmax(prob)
        # print(prob[0][idx])
        # print(labels)
        residuals.append(idx.item() != max_occurence(labels))
        # print(kappa)
        # print(residuals)
        # break
    # break
        
    
# kappa, residuals = sort(kappa, residuals )

In [16]:
kappa = np.array(kappa)
residuals = np.array(residuals)
risk_dict_vec_scal = {}
risk_stars = [0.02, 0.1, 0.15, 0.20, 0.25,0.30, 1.0] # desired risk
delta = 0.001 ## confidence
for desired_risk in risk_stars:
    bound_cal = risk_control()
    [theta, b_star] = bound_cal.bound(desired_risk, delta, kappa, residuals, split= True)
    risk_dict_vec_scal[str(desired_risk)] = [theta, b_star]


0.02 & 0.0000 & 0.0000 & 0.0305 & 0.4579 & 0.9999  \\
0.10 & 0.0957 & 0.6769 & 0.0936 & 0.6746 & 0.1000  \\
0.15 & 0.1452 & 0.7933 & 0.1461 & 0.7934 & 0.1500  \\
0.20 & 0.1950 & 0.9020 & 0.1977 & 0.9016 & 0.2000  \\
0.25 & 0.2449 & 0.9991 & 0.2436 & 0.9990 & 0.2500  \\
0.30 & 0.2458 & 1.0000 & 0.2440 & 1.0000 & 0.2509  \\
1.00 & 0.2447 & 1.0000 & 0.2451 & 0.9999 & 0.2498  \\


In [11]:
risk_dict_vec_scal

{'0.02': [1.0, 0.9999411106109619],
 '0.1': [0.998445, 0.09999037189914387],
 '0.15': [0.9656034, 0.14999588742648362],
 '0.2': [0.77072865, 0.19999776104858114],
 '0.25': [0.2225435, 0.24999369884229788],
 '0.3': [0.1459447, 0.24969663580229495],
 '1.0': [0.14571935, 0.25128186931671803]}

In [12]:

risk_dict = {
    'max_prob' : risk_dict_max_prob,
    'vector_scaling_calibration' : risk_dict_vec_scal
}
path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/risk_bounds/"
torch.save(risk_dict, path + "risk_bound.pt")


In [13]:
np.sum(residuals)

51335

In [25]:
residuals.size

209608

In [26]:
np.sum(residuals)/residuals.size

0.24490954543719706